In [3]:
import pandas as pd
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin

class BaseCSVTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.data = None

        # ensure CSV existsimport pandas as pd
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin

class BaseCSVTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.data = None

        # ensure CSV exists
        if not self.file_path.exists():
            pd.DataFrame().to_csv(self.file_path, index=False)

    def fit(self, X=None, y=None):
        """Load CSV into memory."""
        self.data = pd.read_csv(self.file_path)
        return self

    def transform(self, X=None):
        """Return current dataframe for pipeline usage."""
        if self.data is None:
            self.fit()
        return self.data.copy()

    def save(self):
        """Persist current dataframe to CSV."""
        if self.data is not None:
            self.data.to_csv(self.file_path, index=False)

        if not self.file_path.exists():
            pd.DataFrame().to_csv(self.file_path, index=False)

    def fit(self, X=None, y=None):
        """Load CSV into memory (X,y ignored for sklearn compatibility)."""
        self.data = pd.read_csv(self.file_path)
        return self

    def transform(self, X=None):
        """Return current dataframe for pipeline usage."""
        if self.data is None:
            self.fit()
        return self.data.copy()

    def update(self, new_data: pd.DataFrame, mode="append"):
        """Custom method: append or overwrite data in CSV."""
        if self.data is None:
            self.fit()

        if mode == "append":
            self.data = pd.concat([self.data, new_data], ignore_index=True)
        elif mode == "overwrite":
            self.data = new_data
        else:
            raise ValueError("mode must be 'append' or 'overwrite'")

        self.data.to_csv(self.file_path, index=False)
        return self


In [ ]:
class ProductListTransformer(BaseCSVTransformer):
    def create(self, product_id, product_type, sugar, weight, area, mrp):
        """Add a new product."""
        new_row = pd.DataFrame([{
            "ProductID": product_id,
            "ProductType": product_type,
            "SugarContent": sugar,
            "Weight": weight,
            "Area": area,
            "MRP": mrp
        }])
        self.data = pd.concat([self.transform(), new_row], ignore_index=True)
        self.save()

    def read(self, product_id=None):
        """Fetch all products or one by ID."""
        df = self.transform()
        if product_id:
            return df[df["ProductID"] == product_id]
        return df

    def update(self, product_id, **kwargs):
        """Update product attributes."""
        df = self.transform()
        for key, val in kwargs.items():
            df.loc[df["ProductID"] == product_id, key] = val
        self.data = df
        self.save()

    def delete(self, product_id):
        """Delete a product."""
        df = self.transform()
        df = df[df["ProductID"] != product_id]
        self.data = df
        self.save()


In [ ]:
class CartTransformer(BaseCSVTransformer):
    def create(self, cart_id, product_id, quantity):
        new_row = pd.DataFrame([{
            "CartID": cart_id,
            "ProductID": product_id,
            "Quantity": quantity
        }])
        self.data = pd.concat([self.transform(), new_row], ignore_index=True)
        self.save()

    def read(self, cart_id=None):
        df = self.transform()
        if cart_id:
            return df[df["CartID"] == cart_id]
        return df

    def update(self, cart_id, product_id, quantity):
        df = self.transform()
        df.loc[(df["CartID"] == cart_id) & (df["ProductID"] == product_id), "Quantity"] = quantity
        self.data = df
        self.save()

    def delete(self, cart_id, product_id=None):
        df = self.transform()
        if product_id:
            df = df[~((df["CartID"] == cart_id) & (df["ProductID"] == product_id))]
        else:
            df = df[df["CartID"] != cart_id]
        self.data = df
        self.save()


In [ ]:
class PurchaseTransformer(BaseCSVTransformer):
    def create(self, purchase_id, cart_id, total_amount):
        new_row = pd.DataFrame([{
            "PurchaseID": purchase_id,
            "CartID": cart_id,
            "TotalAmount": total_amount
        }])
        self.data = pd.concat([self.transform(), new_row], ignore_index=True)
        self.save()

    def read(self, purchase_id=None):
        df = self.transform()
        if purchase_id:
            return df[df["PurchaseID"] == purchase_id]
        return df

    def update(self, purchase_id, **kwargs):
        df = self.transform()
        for key, val in kwargs.items():
            df.loc[df["PurchaseID"] == purchase_id, key] = val
        self.data = df
        self.save()

    def delete(self, purchase_id):
        df = self.transform()
        df = df[df["PurchaseID"] != purchase_id]
        self.data = df
        self.save()


In [ ]:
class DiscountTransformer(BaseCSVTransformer):
    def create(self, discount_id, product_id, discount_percent):
        new_row = pd.DataFrame([{
            "DiscountID": discount_id,
            "ProductID": product_id,
            "DiscountPercent": discount_percent
        }])
        self.data = pd.concat([self.transform(), new_row], ignore_index=True)
        self.save()

    def read(self, discount_id=None):
        df = self.transform()
        if discount_id:
            return df[df["DiscountID"] == discount_id]
        return df

    def update(self, discount_id, **kwargs):
        df = self.transform()
        for key, val in kwargs.items():
            df.loc[df["DiscountID"] == discount_id, key] = val
        self.data = df
        self.save()

    def delete(self, discount_id):
        df = self.transform()
        df = df[df["DiscountID"] != discount_id]
        self.data = df
        self.save()


In [ ]:
from flask import Flask, request, jsonify
from product_transformer import ProductCSVTransformer
from cart_transformer import CartCSVTransformer
from purchase_transformer import PurchaseCSVTransformer
from discount_transformer import DiscountCSVTransformer

app = Flask(__name__)

# Instantiate transformers (each works on its own CSV)
product_transformer = ProductCSVTransformer("data/products.csv")
cart_transformer = CartCSVTransformer("data/cart.csv")
purchase_transformer = PurchaseCSVTransformer("data/purchases.csv")
discount_transformer = DiscountCSVTransformer("data/discounts.csv")

# -------------------------------
# Product API
# -------------------------------
@app.route("/products", methods=["GET"])
def get_products():
    return jsonify(product_transformer.read_csv().to_dict(orient="records"))

@app.route("/products", methods=["POST"])
def add_product():
    data = request.json
    product_transformer.create(data)
    return jsonify({"message": "Product added successfully!"}), 201

@app.route("/products/<int:row_id>", methods=["PUT"])
def update_product(row_id):
    data = request.json
    product_transformer.update(row_id, data)
    return jsonify({"message": "Product updated successfully!"})

@app.route("/products/<int:row_id>", methods=["DELETE"])
def delete_product(row_id):
    product_transformer.delete(row_id)
    return jsonify({"message": "Product deleted successfully!"})


# -------------------------------
# Cart API
# -------------------------------
@app.route("/cart", methods=["GET"])
def get_cart():
    return jsonify(cart_transformer.read_csv().to_dict(orient="records"))

@app.route("/cart", methods=["POST"])
def add_to_cart():
    data = request.json
    cart_transformer.create(data)
    return jsonify({"message": "Item added to cart!"}), 201

@app.route("/cart/<int:row_id>", methods=["DELETE"])
def delete_from_cart(row_id):
    cart_transformer.delete(row_id)
    return jsonify({"message": "Item removed from cart!"})


# -------------------------------
# Purchase API
# -------------------------------
@app.route("/purchases", methods=["GET"])
def get_purchases():
    return jsonify(purchase_transformer.read_csv().to_dict(orient="records"))

@app.route("/purchases", methods=["POST"])
def add_purchase():
    data = request.json
    purchase_transformer.create(data)
    return jsonify({"message": "Purchase recorded!"}), 201


# -------------------------------
# Discount API
# -------------------------------
@app.route("/discounts", methods=["GET"])
def get_discounts():
    return jsonify(discount_transformer.read_csv().to_dict(orient="records"))

@app.route("/discounts", methods=["POST"])
def add_discount():
    data = request.json
    discount_transformer.create(data)
    return jsonify({"message": "Discount added!"}), 201

@app.route("/discounts/<int:row_id>", methods=["PUT"])
def apply_discount(row_id):
    data = request.json
    discount_transformer.update(row_id, data)
    return jsonify({"message": "Discount updated!"})


if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
# Products
products = ProductListTransformer("products.csv").fit()
products.create(1, "Chocolate", "High", 200, "IN", 150)
print(products.read())

# Cart
cart = CartTransformer("cart.csv").fit()
cart.create(101, 1, 2)   # cart_id=101, product_id=1, qty=2
print(cart.read(101))

# Purchase
purchase = PurchaseTransformer("purchases.csv").fit()
purchase.create(5001, 101, 300)
print(purchase.read(5001))

# Discount
discount = DiscountTransformer("discounts.csv").fit()
discount.create(9001, 1, 10)  # 10% discount
print(discount.read())


In [ ]:
curl -X POST http://127.0.0.1:5000/products \
     -H "Content-Type: application/json" \
     -d '{"id": 1, "name": "Laptop", "price": 1200}'


In [ ]:
curl http://127.0.0.1:5000/products
